In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8-darkgrid")

# Localisation du fichier CSV
current_dir = Path().resolve()
project_root = current_dir.parent
data_path = project_root / "Data" / "Dataset of weighing station temperature measurements.csv"

print("Fichier CSV :", data_path)

# IMPORTANT : le fichier est séparé par des ';'
df = pd.read_csv(data_path, sep=";")

# Aperçu
display(df.head())
print(df.columns)


In [ ]:
# On convertit la colonne Time en datetime et on la met en index
df['Time'] = pd.to_datetime(df['Time'])
df = df.set_index('Time').sort_index()

df.head()


In [ ]:
# Colonnes de température à l'intérieur du puits
low_cols = [c for c in df.columns if "Low" in c]
mid_cols = [c for c in df.columns if "Mid" in c]
top_cols = [c for c in df.columns if "Top" in c]

# On les trie par numéro de capteur S1, S2, ... S29
def sort_by_sensor(col_list):
    return sorted(col_list, key=lambda x: int(x.split("-S")[1]))

low_cols = sort_by_sensor(low_cols)
mid_cols = sort_by_sensor(mid_cols)
top_cols = sort_by_sensor(top_cols)

print("Nb capteurs LOW :", len(low_cols))
print("Nb capteurs MID :", len(mid_cols))
print("Nb capteurs TOP :", len(top_cols))

print("\nExemples LOW :", low_cols[:5])
print("Exemples MID :", mid_cols[:5])
print("Exemples TOP :", top_cols[:5])


In [ ]:
sensor = 1  # change à 6, 10, etc. pour d'autres poteaux

col_low = low_cols[sensor-1]
col_mid = mid_cols[sensor-1]
col_top = top_cols[sensor-1]

plt.figure(figsize=(12,5))
plt.plot(df.index, df[col_low], label=f"LOW S{sensor}")
plt.plot(df.index, df[col_mid], label=f"MID S{sensor}")
plt.plot(df.index, df[col_top], label=f"TOP S{sensor}")
plt.xlabel("Temps")
plt.ylabel("Température [°C]")
plt.title(f"Température au poteau S{sensor} (LOW / MID / TOP)")
plt.legend()
plt.show()


In [ ]:
delta_z = 0.8  # m entre LOW et TOP

T_low = df[col_low]
T_top = df[col_top]

dT = T_top - T_low               # différence de température TOP - LOW [°C]
grad = dT / delta_z              # gradient vertical [°C/m]

plt.figure(figsize=(12,4))
plt.plot(df.index, grad)
plt.axhline(0, linestyle='--', color='black')
plt.xlabel("Temps")
plt.ylabel("Gradient (TOP-LOW)/0.8 [°C/m]")
plt.title(f"Gradient vertical de température au poteau S{sensor}")
plt.show()


In [ ]:
# ΔT TOP-LOW pour tous les poteaux (S1 à S29)
dT_all = pd.DataFrame({
    f"S{i+1}": df[top_cols[i]] - df[low_cols[i]]
    for i in range(29)
})

grad_all = dT_all / delta_z  # [°C/m]

# Gradient moyen par poteau
mean_grad_per_sensor = grad_all.mean()
overall_mean_grad = grad_all.stack().mean()

print("Gradient moyen par poteau [°C/m]:\n", mean_grad_per_sensor)
print("\nGradient moyen global [°C/m]:", overall_mean_grad)


In [ ]:
threshold = 1.5  # °C

is_stratified = dT_all > threshold  # DataFrame booléen

# Fraction du temps stratifié par poteau
frac_time_per_sensor = is_stratified.mean()

# Fraction du temps où au moins un poteau est stratifié
frac_time_any_sensor = is_stratified.any(axis=1).mean()

print("Fraction du temps stratifié par poteau:\n", frac_time_per_sensor)
print("\nFraction du temps où au moins un poteau est stratifié :", frac_time_any_sensor)


In [ ]:
plt.figure(figsize=(14, 6))

# dT_all : DataFrame de shape (n_times, 29) que tu as déjà
# On transpose pour avoir poteaux sur l'axe Y
plt.imshow(dT_all.T, aspect='auto', cmap='turbo',
           extent=[0, dT_all.shape[0], 1, 29])

plt.colorbar(label="ΔT = T_top - T_low [°C]")
plt.xlabel("Index temporel (échantillons)")
plt.ylabel("Poteau (S1 → S29)")
plt.title("Heatmap de la stratification ΔT (TOP - LOW) sur tous les poteaux")
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))

plt.imshow(grad_all.T, aspect='auto', cmap='turbo',
           extent=[0, grad_all.shape[0], 1, 29])

plt.colorbar(label="Gradient vertical [°C/m]")
plt.xlabel("Index temporel (échantillons)")
plt.ylabel("Poteau (S1 → S29)")
plt.title("Heatmap du gradient vertical de température (tous les poteaux)")
plt.show()


In [ ]:
# Si nécessaire : reconstruire dT_all
dT_all = pd.DataFrame({
    f"S{i+1}": df[top_cols[i]] - df[low_cols[i]]
    for i in range(29)
})

# Moyenne quotidienne
dT_daily = dT_all.resample("D").mean()
dT_daily.tail()

In [ ]:
plt.figure(figsize=(14, 6))

plt.imshow(dT_daily.T,
           aspect='auto',
           cmap='turbo',
           extent=[0, dT_daily.shape[0], 1, 29])

plt.colorbar(label="ΔT moyen par jour [°C]")
plt.xlabel("Jour (index)")
plt.ylabel("Poteau S1 → S29")
plt.title("Heatmap quotidienne de ΔT (TOP - LOW)")

plt.show()


In [ ]:
delta_z = 0.8
grad_all = dT_all / delta_z

grad_daily = grad_all.resample("D").mean()

plt.figure(figsize=(14, 6))

plt.imshow(grad_daily.T,
           aspect='auto',
           cmap='turbo',
           extent=[0, grad_daily.shape[0], 1, 29])

plt.colorbar(label="Gradient vertical moyen [°C/m]")
plt.xlabel("Jour (index)")
plt.ylabel("Poteau S1 → S29")
plt.title("Heatmap quotidienne du gradient vertical de température")

plt.show()


In [ ]:

# --- 1. ΔT TOP-LOW pour tous les poteaux --- ###
dT_all = pd.DataFrame({
    f"S{i+1}": df[top_cols[i]] - df[low_cols[i]]
    for i in range(29)
})

# Moyenne temporelle de la stratification par poteau
mean_dT = dT_all.mean()   # Série indexée par 'S1'...'S29'

# --- 2. Positions XY des poteaux (à adapter si tu veux le vrai plan) --- #
sensor_names = [f"S{i+1}" for i in range(29)]

# Grille simple 10x3 pour avoir un plan 2D lisible (placeholder)
sensor_positions = {}
for i, name in enumerate(sensor_names):
    x = i % 10       # colonne
    y = i // 10      # rangée
    sensor_positions[name] = (x, y)

# --- 3. Construire les tableaux x, y, couleur --- #
xs = [sensor_positions[s][0] for s in sensor_names]
ys = [sensor_positions[s][1] for s in sensor_names]
cs = [mean_dT[s] for s in sensor_names]

plt.figure(figsize=(8, 4))
sc = plt.scatter(xs, ys, c=cs, s=250, cmap="turbo")
plt.colorbar(sc, label="ΔT moyen (TOP-LOW) [°C]")

# Ajouter les labels S1, S2...
for s, x, y in zip(sensor_names, xs, ys):
    plt.text(x, y, s, ha="center", va="center", color="black", fontsize=8, weight="bold")

plt.gca().set_aspect("equal")
plt.title("Carte 2D des poteaux colorée par la stratification moyenne ΔT (TOP-LOW)")
plt.xlabel("X (position arbitraire)")
plt.ylabel("Y (position arbitraire)")
plt.grid(True, alpha=0.3)
plt.show()
